In [ ]:
import torch
from faker import Faker
from itertools import product

# General parameters
SEED = 42
INFERENCE_DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset parameters
NUM_SAMPLES = 10000
SCENARIO_NAME = "city_6_miami"
BS_IDX = 1

# Simulation parameters
P_TOTAL = 20.0
NOISE_VARIANCE = 1e-12

# LWM Model Parameters
D_MODEL = 128
MODEL_PATH = "./models/model.pth"

# Training Parameters
EPOCHS = 25
WARMUP_EPOCHS = 10
LEARNING_RATE = 1e-3
FINETUNE_LEARNING_RATE = 1e-5

optimizer_configs = {"task_head_lr": LEARNING_RATE,
                    "encoder_lr": FINETUNE_LEARNING_RATE}

NUM_USERS = [4, 8, 12, 16]
PATCH_SIZE = [
    {"patch_cols": 2, "patch_rows": 2},
    {"patch_cols": 4, "patch_rows": 4},
    {"patch_cols": 16, "patch_rows": 1}, # One Token per carrier
    ]
TRAINING_RATIOS = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
HIDDEN_DIMS = [64, 128, 256, 512, 1024]
FINETUNE_STATUS = [None, ["layers.8", "layers.9", "layers.10", "layers.11"], "full"]
INPUT_TYPES = ["cls", "channel_emb", "raw"]
PRECODERS = ["mrt", "zf"]

experiments = [
    {
        "num_users": num_users,
        "patch_rows": patch["patch_rows"],
        "patch_cols": patch["patch_cols"],
        "training_ratio": train_ratio,
        "hidden_dim": hidden_dim,
        "finetune": finetune,
        "input_type": input_type,
        "precoder": precoder
    }
    for num_users, patch, train_ratio, hidden_dim, finetune, input_type, precoder
    in product(
        NUM_USERS,
        PATCH_SIZE,
        TRAINING_RATIOS,
        HIDDEN_DIMS,
        FINETUNE_STATUS,
        INPUT_TYPES,
        PRECODERS
    )
]

# Initialize with the German locale
fake = Faker('de_DE')

In [5]:
import json
from src.dataset import PowerAllocationDataset
from src.utils import prepare_loaders, get_subset
from src.lwm_model import lwm
from src.downstream_models import PowerAllocator, Wrapper
from src.metrics import SumRateLoss, Benchmark, EmbeddingAnalyzer
from src.train import train_downstream_model, finetune

for parameters in experiments:
    USERS = parameters['num_users']
    PATCH_ROWS = parameters["patch_rows"]
    PATCH_COLS = parameters["patch_cols"]
    TRAINING_RATIO = parameters["training_ratio"]
    HIDDEN_DIM = parameters["hidden_dim"]
    FINETUNE = parameters["finetune"] if INPUT_TYPE != "raw" else None
    INPUT_TYPE = parameters["input_type"]
    PRECODER = parameters["precoder"]
    
    experiment_name = fake.city()
    results_folder = f"./results/{experiment_name}"

    dataset = PowerAllocationDataset(num_samples=NUM_SAMPLES,
                                     num_users=USERS,
                                     scenario_name=SCENARIO_NAME,
                                     bs_idx=BS_IDX,
                                     patch_cols=PATCH_COLS,
                                     patch_rows=PATCH_ROWS)
    
    NUM_SUBCARRIERS = dataset.generator.n_subcarriers
    
    train_loader, val_loader, test_loader = prepare_loaders(dataset.raw_channels, dataset.data_tokens, seed=SEED)
    fraction_train_loader = get_subset(train_loader, TRAINING_RATIO, seed=SEED)

    lwm_model = lwm.from_pretrained(ckpt_name=MODEL_PATH, device=INFERENCE_DEVICE)
    if INPUT_TYPE == "raw":
        task_head = PowerAllocator(32, num_subcarriers=NUM_SUBCARRIERS, hidden_dim=HIDDEN_DIM)
    else:
        task_head = PowerAllocator(D_MODEL, num_subcarriers=NUM_SUBCARRIERS, hidden_dim=HIDDEN_DIM)

    model = Wrapper(lwm_model, task_head, input_type=parameters["input_type"]).to(INFERENCE_DEVICE)

    criterion = SumRateLoss(P_TOTAL, NOISE_VARIANCE, precoder=PRECODER)

    trackio_params = {
        "project": "power_allocation",
        "group": PRECODER,
        "config": {
            "epochs": EPOCHS,
            "device": INFERENCE_DEVICE,
            "p_total": P_TOTAL,
            "noise_variance": NOISE_VARIANCE
        }
    }
    if parameters["finetune"] == None or parameters["input_type"] == "raw":
        model= train_downstream_model(model=model,
                                      train_loader=fraction_train_loader,
                                      val_loader=val_loader,
                                      optimizer_configs=optimizer_configs,
                                      criterion=criterion,
                                      epochs=EPOCHS,
                                      device=INFERENCE_DEVICE,
                                      save_dir=results_folder,
                                      trackio_params=trackio_params)
    else:
        model = finetune(model=model,
                         train_loader=fraction_train_loader,
                         val_loader=val_loader,
                         fine_tune_layers=FINETUNE,
                         optimizer_configs=optimizer_configs,
                         criterion=criterion,
                         epochs=EPOCHS,
                         warmup_epochs=WARMUP_EPOCHS,
                         device=INFERENCE_DEVICE,
                         save_dir=results_folder,
                         trackio_params=trackio_params)
        
    with open(f"{results_folder}/parameters.json", "w") as f:
        json.dump(parameters, f, indent=4)

    benchmark = Benchmark(test_loader, P_TOTAL, NOISE_VARIANCE, precoder=PRECODER)
    benchmark_results = benchmark.evaluate(model)
    benchmark.plot_cdf(benchmark_results, results_folder)
    if INPUT_TYPE != "raw":
        analyzer = EmbeddingAnalyzer(model, INFERENCE_DEVICE)
        analyzer.run(test_loader, results_dir=results_folder)


Basestation 1

UE-BS Channels


Generating Scenarios: 100%|██████████| 10000/10000 [00:11<00:00, 881.94it/s]


Model loaded successfully from ./models/model.pth
* Created new run: Run_2025-12-27_00-53


 80%|████████  | 20/25 [00:17<00:04,  1.11it/s, Train Loss=-18.9, Validation Loss=-18.5, LR=0.001]


KeyboardInterrupt: 